In [17]:
import pandas as pd

In [18]:
concentration = pd.read_csv('../../data/2019/19_concentrations.csv')
labels = pd.read_csv('../../data/2019/19_yellow_labels.csv')
pit = pd.read_csv('../../data/2019/19_time_since_pit.csv')
weather = pd.read_csv('../../data/weather_data/2019_weather.csv', sep=';')

In [19]:
max_concentration = pd.DataFrame(columns=['Time', 'Max Concentration'])
temp_max = 0
for index, row in concentration.iterrows():
    sector = row[1]
    if sector == 1:
        temp_max = 0
    if row[2] > temp_max:
        temp_max = row[2]
    if sector == 13:
        max_concentration.loc[index] = [row[0], temp_max]

In [33]:
start_time = weather['TIME_UTC_SECONDS'].iloc[0]

with open('../../data/2019/19_weather_mod.csv', 'w') as f:
    f.write('Time,Air Temp,Track Temp,Humidity,Pressure,Wind Speed\n')
    for index, row in weather.iterrows():
        if 65299 < ((row[0]-start_time)//20 * 20) < 65461:
            f.write("%s,%s,%s,%s,%s,%s\n"%((row[0]-start_time)//20 * 20, 55.5,row[3], row[4], row[5], row[6]))
            f.write("%s,%s,%s,%s,%s,%s\n"%((row[0]-start_time+20)//20 * 20,55.5,row[3], row[4], row[5], row[6]))
            f.write("%s,%s,%s,%s,%s,%s\n"%((row[0]-start_time+40)//20 * 20,55.5,row[3], row[4], row[5], row[6]))
        else:
            f.write("%s,%s,%s,%s,%s,%s\n"%((row[0]-start_time)//20 * 20, row[2],row[3], row[4], row[5], row[6]))
            f.write("%s,%s,%s,%s,%s,%s\n"%((row[0]-start_time+20)//20 * 20,row[2],row[3], row[4], row[5], row[6]))
            f.write("%s,%s,%s,%s,%s,%s\n"%((row[0]-start_time+40)//20 * 20,row[2],row[3], row[4], row[5], row[6]))
weather_mod = pd.read_csv('../../data/2019/19_weather_mod.csv')

In [41]:
print(pit.columns)
pit = pit[pit['Last Pit'] < 130]
agg = max_concentration.merge(pit, how='inner', on='Time')
agg = agg.merge(weather_mod, how='inner', on='Time')
agg = agg.merge(labels, how='inner', on='Time')
pit

Index(['Time', 'Last Pit'], dtype='object')


,Time,Last Pit
0,20,0.333333
1,40,0.666667
2,60,1.000000
3,80,1.333333
4,100,1.666667
...,...,...
3948,79200,55.860050
3949,79220,56.193383
3950,79240,56.526717
3951,79260,56.860050


In [42]:
agg.to_csv('../../data/2019/19_agg_data.csv', index=False)